In [3]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-apple-darwin
info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: latest update on 2024-10-17, rust version 1.82.0 (f6e511eec 2024-10-15)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
 59.6 MiB /  59.6 MiB (100 %)  33.0 MiB/s in  1s ETA:  0s
info: downloading component 'rustfmt'
info: installing component 'cargo'
  7.1 MiB /   7.1 MiB (100 %)   4.9 MiB/s in  1s ETA:  0s
info: installing component 'clippy'
info: installing component 'rust-docs'
 16.3 MiB /  16.3 MiB (100 %)   1.9 MiB/s in 14s ETA:  0s    
info: installing component 'rust-std'
 24.9 MiB /  24.9 MiB (100 %)   8.6 MiB/s in  2s ETA:  0s
info: installing component 'rustc'
 59.6 MiB /  59.6 MiB (100 %)   9.3 MiB/s in  6s ETA:  0s
info: installing component 'rustfmt'
info: defau

In [5]:
import os
os.environ['PATH'] += os.pathsep + os.path.expanduser('~/.cargo/bin')

In [7]:
!rustc --version
!cargo --version


rustc 1.82.0 (f6e511eec 2024-10-15)
cargo 1.82.0 (8f40fc59f 2024-08-21)


In [9]:
!pip install "quri-parts[braket,cirq,qiskit,qulacs,tket]"


  Using cached quri_parts-0.20.3-py3-none-any.whl.metadata (6.6 kB)
  Using cached quri_parts_algo-0.20.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached quri_parts_braket-0.20.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached quri_parts_chem-0.20.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached quri_parts_circuit-0.20.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached quri_parts_cirq-0.20.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached quri_parts_core-0.20.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached quri_parts_qiskit-0.20.3-py3-none-any.whl.metadata (1.3 kB)
  Using cached quri_parts_qulacs-0.20.3-py3-none-any.whl.metadata (1.3 kB)
  Using cached quri_parts_tket-0.20.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached amazon_braket_schemas-1.22.2-py3-none-any.whl.metadata (5.8 kB)
  Using cached amazon_braket_sdk-1.88.1-py3-none-any.whl.metadata (14 kB)
  Using cached quri_parts_rust-0.20.3.tar.gz (629 kB)
  Installing build dependencies ... done
  Getting requirem

In [13]:
import sys
print("Pythonのバージョン:", sys.version)


Pythonのバージョン: 3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:14:12) [Clang 14.0.6 ]


In [83]:
import numpy as np
from quri_parts.circuit.gates import PauliRotation
from collections.abc import Sequence
from typing import Optional, Sequence, List, Tuple

from quri_parts.circuit import (
    ImmutableLinearMappedParametricQuantumCircuit,
    LinearMappedParametricQuantumCircuit,
)

from quri_parts.algo.ansatz.two_local import (
    EntanglementPatternType,
    EntLayerMakerArg,
    RotLayerMakerArg,
    TwoLocal,
    build_entangler_map,
)

def Rxx_gate(index, theta):
    gate = PauliRotation(index, (1, 1), theta) #X gate: pauli_ids=1
    return gate

def _add_rz_gates(circuit, arg):
    layer_index, qubit_index = arg
    theta = circuit.add_parameter(f"theta_{layer_index}_{qubit_index}")
    circuit.add_ParametricRZ_gate(qubit_index, theta)
    

def _add_rxx_rz_gates(circuit, arg):
    layer_index, (i, j) = arg
    circuit.add_gate(Rxx_gate((i, j), np.pi / 2))  
    circuit.add_ParametricRZ_gate(i, circuit.add_parameter(f"theta_{layer_index}_{i}"))  
    circuit.add_ParametricRZ_gate(j, circuit.add_parameter(f"theta_{layer_index}_{j}"))  
    circuit.add_gate(Rxx_gate((i, j), -np.pi / 2))  

In [118]:
class Ansatz(ImmutableLinearMappedParametricQuantumCircuit):
    _add_rotation_gates = _add_rz_gates
    _add_entanglement_gates = _add_rxx_rz_gates

    def __init__(
        self,
        qubit_count: int,
        depth: int,
        rotation_indices: Optional[Sequence[int]] = None,
        entangler_map_seq: Optional[Sequence[Sequence[Tuple[int, int]]]] = None,
    ):
        if rotation_indices is None:
            rotation_indices = list(range(qubit_count))

        if entangler_map_seq is None:
            entangler_map_seq = self._build_entangler_map_seq(qubit_count, depth)

        circuit = TwoLocal(
            qubit_count,
            layer_pattern="e" * 2 * depth,
            rot_layer_maker=self.__class__._add_rotation_gates,
            ent_layer_maker=self.__class__._add_entanglement_gates,
            rotation_indices=rotation_indices,
            entangler_map_seq=entangler_map_seq,
        )
        super().__init__(circuit)
        #self.depth = depth

    def _build_entangler_map_seq(self, qubit_count, depth):
        entangler_map_seq =[]

        for i in range(2 * depth):
            if i % 2 == 0:
                layer = [(j, j + 1) for j in range(0, qubit_count - 1, 2)]
            elif i % 2 == 1:
                layer = [(j, j + 1) for j in range(1, qubit_count - 1, 2)]
        
            entangler_map_seq.append(layer)
            
        
        return entangler_map_seq   


In [122]:
from quri_parts.circuit.utils.circuit_drawer import draw_circuit
ansatz = Ansatz(qubit_count=6, depth=1)
draw_circuit(ansatz)

   ___     ___     ___                          
  |PR |   |PRZ|   |PR |                         
--|0  |---|1  |---|3  |-------------------------
  |   |   |___|   |   |                         
  |   |    ___    |   |    ___     ___     ___  
  |   |   |PRZ|   |   |   |PR |   |PRZ|   |PR | 
--|   |---|2  |---|   |---|12 |---|13 |---|15 |-
  |___|   |___|   |___|   |   |   |___|   |   | 
   ___     ___     ___    |   |    ___    |   | 
  |PR |   |PRZ|   |PR |   |   |   |PRZ|   |   | 
--|4  |---|5  |---|7  |---|   |---|14 |---|   |-
  |   |   |___|   |   |   |___|   |___|   |___| 
  |   |    ___    |   |    ___     ___     ___  
  |   |   |PRZ|   |   |   |PR |   |PRZ|   |PR | 
--|   |---|6  |---|   |---|16 |---|17 |---|19 |-
  |___|   |___|   |___|   |   |   |___|   |   | 
   ___     ___     ___    |   |    ___    |   | 
  |PR |   |PRZ|   |PR |   |   |   |PRZ|   |   | 
--|8  |---|9  |---|11 |---|   |---|18 |---|   |-
  |   |   |___|   |   |   |___|   |___|   |___| 
  |   |    ___    | 